## TensorFlow 2.2 MobileNet 模型部署

In [10]:
#!pip install tensorflow==2.2.0rc2

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [2]:
from tensorflow import keras as keras

keras.backend.clear_session()  # For easy reset of notebook state.

In [3]:
# model_path = "saved_model/mobilenet"
model_path = "mobilenet_dog.h5"

model = keras.models.load_model(model_path)

In [5]:
#model.summary()

### 创建工作区（Workspace）

In [6]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

notebook-gpu
django-test
japaneast
b8cd4096-cfec-4e20-8b90-eddc24402703


In [7]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='tf2-mobilenet',
                       model_path=model_path,
                       model_framework=Model.Framework.TENSORFLOW,  # Framework used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=4))

print('Name:', model.name)
print('Version:', model.version)

Registering model tf2-mobilenet
Name: tf2-mobilenet
Version: 4


In [8]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment(name = 'tf2-env')
inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="deploy/score.py",
                                   conda_file="deploy/infer_env.yml")


In [9]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.exceptions import WebserviceException


aks_target = AksCompute(ws, "infer-aks")
service_name='tf2-mobilenet'

try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass


deployment_config = AksWebservice.deploy_configuration()


service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=deployment_config, 
                       deployment_target=aks_target)

service.wait_for_deployment(True)
print(service.state)

Running......
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [10]:
import requests
import json

# Retrieve the API keys. Two keys were generated.
key1, Key2 = service.get_keys()
print(key1)

data = json.dumps({'data': "http://www.mgous.com/img/p/1905/28144928349/1_620_462.jpg"})
data = bytes(data, encoding='utf8')

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(service.scoring_uri, data=data, headers=headers)

print("POST to url", service.scoring_uri)
print(resp.text)

7oOBKj6LhXfhnxiruMacaATcYXh88e0e
POST to url http://52.243.41.251:80/api/v1/service/tf2-mobilenet/score
"{\"result\": [[0.0, 1.0]]}"


In [11]:
service.delete()